In [1]:
import pyvisa
import time
import schedule
import pandas as pd
import serial.tools.list_ports
import os

Port_Arduino = 'COM10'

# Destination folder and file path
folder_path = "C:\\Users\\kary\\Documents\\9_data\\IEAP\\"
file_path = os.path.join(folder_path, "voltage_data.csv")

#Create Resource manager instance
rm = pyvisa.ResourceManager()
#Check the VISA device
rm.list_resources()

('USB0::0x05E6::0x6500::04537333::INSTR', 'ASRL10::INSTR')

In [2]:
# Start the communication by VISA identifying resource name
dmm = rm.open_resource('USB0::0x05E6::0x6500::04537333::INSTR')

#Checking the machine name
print(dmm.query('*IDN?'))

KEITHLEY INSTRUMENTS,MODEL DMM6500,04537333,1.7.11b



In [11]:
dmm.write('*RST')
dmm.write(':DISPlay:VOLTage:DIGits 3')

27

In [12]:
# Portrate setting
baudrate = 115200

if Port_Arduino is None:
    print("Selected COM port not found.")
else:
    # Make serial port instance
    try:
        seriallInst = serial.Serial(Port_Arduino, baudrate, timeout=1) 
        print("Serial port opened successfully.")
    except serial.SerialException as e:
        print(f"Error opening serial port: {e}")
        exit()

Serial port opened successfully.


### Without Position

In [8]:
#列を2つもつデータフレームに新しい行を追加する
def add_row(df,row):
    new_row = pd.Series({df.columns[0]:row[0],df.columns[1]:row[1]}).to_frame().T
    return  pd.concat([df,new_row],ignore_index = True)

In [9]:
#データフレーム、マルチメータ、合計測定時間、サンプル周期
def df_add_read_data(df,mul,total_read_time,sample_period):
    #測定時間の初期化
    start_time =  time.perf_counter()
    end_time = time.perf_counter()
    while end_time - start_time < total_read_time:
        each_start_time =  time.perf_counter()
        read_data = float(mul.query("READ?;"))
        each_end_time = time.perf_counter()
        #サンプル周期になるまで待機
        while  each_end_time - each_start_time <= sample_period:
            each_end_time = time.perf_counter()

        df = add_row(df,[each_end_time - start_time ,read_data])
        end_time = time.perf_counter()
    return df

In [10]:
columns = ["Time[s]","Measured_Voltage[V]"]
df_voltage = pd.DataFrame(columns=columns)

position = 0
read_time = 5 # sec
freq = 0.001
df_voltage = df_add_read_data(df_voltage,dmm,read_time,freq)
df_voltage

C:\Users\kary\AppData\Local\Temp\ipykernel_15788\3162089667.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return  pd.concat([df,new_row],ignore_index = True)


,Time[s],Measured_Voltage[V]
0,0.110858,-0.000049
1,0.134779,-0.000046
2,0.157287,-0.000030
3,0.180116,-0.000052
4,0.224242,-0.000054
...,...,...
180,4.898977,-0.000082
181,4.921495,-0.000099
182,4.943759,-0.000130
183,4.966664,-0.000069


### Position

In [5]:
#列を3つもつデータフレームに新しい行を追加する
def add_row(df,row):
    new_row = pd.Series({df.columns[0]:row[0],df.columns[1]:row[1],df.columns[2]:row[2]}).to_frame().T
    return  pd.concat([df,new_row],ignore_index = True)

In [6]:
#データフレーム、マルチメータ、合計測定時間、サンプル周期, position
def df_add_read_data(df,mul,total_read_time,sample_period,position):
    #測定時間の初期化
    start_time =  time.perf_counter()
    end_time = time.perf_counter()
    while end_time - start_time < total_read_time:
        each_start_time =  time.perf_counter()
        read_data = float(mul.query("READ?;"))
        each_end_time = time.perf_counter()
        #サンプル周期になるまで待機
        while  each_end_time - each_start_time <= sample_period:
            each_end_time = time.perf_counter()

        #serial read from Arduino
        position = seriallInst.readline().decode('utf-8').strip()
        df = add_row(df,[each_end_time - start_time ,read_data, position])
        end_time = time.perf_counter()
    return df

In [7]:
columns = ["Time[s]","Measured_Voltage[V]","Position[mm]"]
df_voltage = pd.DataFrame(columns=columns)

position = 0
read_time = 5 # sec
freq = 0.001
df_voltage = df_add_read_data(df_voltage,dmm,read_time,freq,position)
df_voltage

,Time[s],Measured_Voltage[V],Position[mm]
0,0.021795,-0.000086,
1,1.047146,-0.000091,
2,2.078928,-0.000093,
3,3.108717,-0.000051,
4,4.141315,-0.000082,


### Memo

In [6]:
# Reset
dmm.write('*RST')

# Beep
dmm.write(':SYSTEM:BEEPer 500, 1')

# Zero
dmm.write('FUNC "VOLT"')
dmm.write('AZER:ONCE')

# Voltage read setting
dmm.write(':SENSe:FUNCtion "VOLTage"')

# Set impedance
dmm.write(':DIG:VOLT:RANG 100e-3') # select 100 mV range

# Read Voltage
print(float(dmm.query('Read?')))

-4.153531e-05
